In [1]:
import os
import pandas as pd 
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSCanonical, PLSRegression, CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, cross_val_score
import matplotlib  as mpl
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
%matplotlib inline
import os, shutil, glob
from PIL import Image
from itertools import cycle
from random import randint
import re, math
import seaborn as sns; sns.set_style("white")
from sklearn.manifold import TSNE
import umap.umap_ as umap
import datetime
import gc
from pathlib import Path

os.getcwd()

'/Users/chririn/Documents/Code/2D_analysis_v2'

### Read data

In [2]:
BaseDir = '/Volumes/data/cellprofiler/automation/results/'
# sorted(os.listdir(BaseDir))

### Input directories

In [3]:
InputFolders = []
NameContains = input("Enter part of name(s) of the folders: ") # e.g.: TX-.*GR-v2
for f in (f for f in os.listdir(BaseDir) if re.match(NameContains, f)):
    InputFolders.append(f)
InputFolders.sort()
print ('InputFolders: ' + str(InputFolders))

InputFolders: ['.DS_Store', '.ipynb_checkpoints', '001_A', '001_B', '002_A', '002_B', '003_A', '003_B', '004_A', '004_B', '005_A', '005_B', '006_A', '006_B', '007_A', '007_B', '008_A', '008_B', '009_A', '009_B', '010_A', '010_B', '011_A', '011_B', '012_A', '012_B', '013_A', '013_B', '014_A', '014_B', '015_A', '015_B', '016_A', '016_B', '190124-U2OS-20X-CopyAP009069', '190131-U2OS-20X-CopyAP009064', '190131-U2OS-20X-CopyAP009065', '190131-U2OS-20X-CopyAP009066', '190131-U2OS-20X-CopyAP009067', '190131-U2OS-20X-CopyAP009068', '190201-U2OS-20X-CopyAP009063', '190201-U2OS-20X-CopyAP009083', '190206-U2OS-20X-CopyAP009070', '190206-U2OS-20X-CopyAP009071', '190206-U2OS-20X-CopyAP009072', '190207-U2OS-20X-CopyAP009073', '190207-U2OS-20X-CopyAP009074', '190207-U2OS-20X-CopyAP009075', '190207-U2OS-20X-CopyAP009076', '190207-U2OS-20X-CopyAP009077', '190207-U2OS-20X-CopyAP009078', '190207-U2OS-20X-CopyAP009079', '190208-U2OS-20X-CopyAP009080', '190208-U2OS-20X-CopyAP009081', '190208-U2OS-20X-CopyA

### Output directory

In [4]:
OutputDir = input("Enter name for output folder: ") # e.g.: ImageMeanFeatures
if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2022-10-21 15:04:12


In [ ]:
groupedbyImageAllPlates = pd.DataFrame()

for oneplate in InputFolders[:]:

        DataFrameDictionary = {}
        ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm', 'featICF_Image']
        for obj in ObjectList:
            DataFrameDictionary[obj] = pd.DataFrame()
            ReadingFile = 0
            #for InpF in InputFolders:
            DataFromFolder =  pd.DataFrame()
            feature_files = glob.glob(BaseDir + '/*' + oneplate + '*/*/*/' + obj + '.csv', recursive=True)
          
            
            
            for file in feature_files[-1:]:
                print(file)
                DataFromOneFile =  pd.read_csv(file, sep=",", low_memory=False)
                print(DataFromOneFile.shape)
                ReadingFile += 1    
                DataFromFolder = DataFromFolder.append(DataFromOneFile.iloc[:,:], ignore_index=True)
                print(DataFromFolder.shape)

                DataFrameDictionary[obj] = DataFrameDictionary[obj].append(DataFromFolder.iloc[:,:], ignore_index=True)
            DataFrameDictionary[obj].columns = [str(col) + '_' + re.sub('_.*', '', re.sub('featICF_', '', obj)) for col in DataFrameDictionary[obj]]
            print('Found %s files' %(ReadingFile))
            print ('Dataframe contains {} columns and {} rows.'.format(DataFrameDictionary[obj].shape[1],
                                                                  DataFrameDictionary[obj].shape[0]))

        df = DataFrameDictionary['featICF_nuclei'].merge(DataFrameDictionary['featICF_cells'], left_on = [ 'Metadata_Barcode_nuclei',
        'Metadata_Site_nuclei', 'Metadata_Well_nuclei','Parent_cells_nuclei'],
                           right_on = [ 'Metadata_Barcode_cells',
        'Metadata_Site_cells', 'Metadata_Well_cells','ObjectNumber_cells'], how='left')
        print(df.shape)

        df = df.merge(DataFrameDictionary['featICF_cytoplasm'], left_on = [ 'Metadata_Barcode_nuclei',
        'Metadata_Site_nuclei', 'Metadata_Well_nuclei','Parent_cells_nuclei'],
                           right_on = [ 'Metadata_Barcode_cytoplasm',
        'Metadata_Site_cytoplasm', 'Metadata_Well_cytoplasm','ObjectNumber_cytoplasm'], how='left')
        print(df.shape)
        df.dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        print(df.shape)
        print('Input: {}'.format(NameContains))
        print('Output: {}'.format(OutputDir))
        print('Dataset shape: {}'.format(df.shape))
        
        del DataFrameDictionary
        gc.collect()
        
        NrOfObjects = df.shape[0]

        Wells = sorted(list(set(df['Metadata_Well_nuclei'])))
        NrOfWells = len(Wells)
        Rows = sorted(list(set([w[0] for w in Wells])))
        print(*Rows)
        NrOfRows = len(Rows)
        Columns = sorted(list(set([w[1:] for w in Wells])))
        NrOfColumns = len(Columns)
        print(*Columns)
        Sites = sorted(list(set(df['Metadata_Site_nuclei'])))
        NrOfSites = len(Sites)
        # print('Plate complete: {}'.format(NrOfRows*NrOfColumns==NrOfWells))
        print('Number of sites: {}'.format(NrOfSites))
        AllWells =[]
        for R in Rows:
            for C in Columns:
                RC = R + C
                # print(RC)
                AllWells += [RC]
        # print(len(AllWells))
        print('Number of wells: {} (full plate is {})'.format(NrOfWells, len(AllWells)))
        
        df['ImageID'] =  df['Metadata_Barcode_nuclei'] + '_' + df['Metadata_Well_nuclei'] + '_' + df['Metadata_Site_nuclei'].astype(str)

        numeric_columns = list()
        for a in df.columns:
            if (df.dtypes[a] == 'float64') | (df.dtypes[a] == 'int64') :
                numeric_columns.append(a)
        print('Found {} numeric columns'.format(len(numeric_columns)))

        dictOfnumeric = { i : np.nanmean for i in numeric_columns}

        groupedbyImage = df.groupby('ImageID').agg(dictOfnumeric)
        groupedbyImage.to_csv('{}/ImageMeansPlate{}.csv'.format(OutputDir, oneplate), sep=';')
        frames = [groupedbyImageAllPlates, groupedbyImage]
        groupedbyImageAllPlates = pd.concat(frames)
        print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
        print('Now the shape of image dataset is: {}'.format(groupedbyImageAllPlates.shape))
        print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
        del df
        del groupedbyImage
        gc.collect()
        
groupedbyImageAllPlates.to_csv('{}/ImageMeansAllPlates.csv'.format(OutputDir), sep=';')

/Volumes/data/cellprofiler/automation/results/colo44-v1-FA-P017365-CACO2-48h-P1-L1/1610/2543/featICF_nuclei.csv
(400758, 754)


/var/folders/bx/43bwk4n57dj9446w1ddn_7t40000gn/T/ipykernel_34160/154669284.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DataFromFolder = DataFromFolder.append(DataFromOneFile.iloc[:,:], ignore_index=True)


(400758, 754)


/var/folders/bx/43bwk4n57dj9446w1ddn_7t40000gn/T/ipykernel_34160/154669284.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DataFrameDictionary[obj] = DataFrameDictionary[obj].append(DataFromFolder.iloc[:,:], ignore_index=True)


Found 1 files
Dataframe contains 754 columns and 400758 rows.
/Volumes/data/cellprofiler/automation/results/colo44-v1-FA-P017365-CACO2-48h-P1-L1/1610/2543/featICF_cells.csv
(272524, 754)


...

Found 1 files
Dataframe contains 754 columns and 220658 rows.
/Volumes/data/cellprofiler/automation/results/colo44-v1-FA-P017368-CACO2-48h-P2-L2/1613/2545/featICF_cytoplasm.csv
(220656, 745)


/var/folders/bx/43bwk4n57dj9446w1ddn_7t40000gn/T/ipykernel_34160/154669284.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DataFromFolder = DataFromFolder.append(DataFromOneFile.iloc[:,:], ignore_index=True)


(220656, 745)


/var/folders/bx/43bwk4n57dj9446w1ddn_7t40000gn/T/ipykernel_34160/154669284.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DataFrameDictionary[obj] = DataFrameDictionary[obj].append(DataFromFolder.iloc[:,:], ignore_index=True)


Found 1 files
Dataframe contains 745 columns and 220656 rows.
(325979, 1508)
(325979, 2253)
(324150, 2253)
Input: colo44
Output: colo44
Dataset shape: (324150, 2253)
B C D E F G H I J K L M N O
02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23
Number of sites: 9
Number of wells: 308 (full plate is 308)
Found 2187 numeric columns
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Now the shape of image dataset is: (11055, 2187)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


In [ ]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2022-10-21 15:50:37
